##### Executing Other SQL Server Queries

In [6]:
import pyodbc
import csv

In [7]:
# Trusted Connection to SQL Server Default Instance with Exception Handling (invalid DB Name)
try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=MSI;DATABASE=sampleDB;Trusted_Connection=yes;')

    # initialize the cursor object with connection name
    cursor = connection.cursor()
    print()
    print('Successfully Connected to Database')
    print()

except pyodbc.Error as ex: # ex is the exception object
    print()
    print('Exception: ', ex)
    print('Stopping program...')
    print()  



Successfully Connected to Database



##### Print the contents of a csv file

In [8]:
with open('sample-csv-file-for-demo.csv',newline='') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in csvReader:
        print(', '.join(row))

1, customer1, firstName1, lastName1
2, customer2, firstName2, lastName2
3, customer3, firstName3, lastName3
4, customer4, firstName4, lastName4
5, customer5, firstName5, lastName5


##### Read Specific Columns for Every Record

In [9]:
with open('sample-csv-file-for-demo.csv',newline='') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in csvReader:
        print(f'First Column of Row: {row[0]}')

First Column of Row: 1
First Column of Row: 2
First Column of Row: 3
First Column of Row: 4
First Column of Row: 5


##### Writing Contents of CSV File to SQL Server Table </br> 1. Create Table and commit Changes </br> 2. For every row of CSV </br>&nbsp;&nbsp;&nbsp;&nbsp;2a. Populate Parameter values </br>&nbsp;&nbsp;&nbsp;&nbsp;2b. Execute the INSERT statement </br>&nbsp;&nbsp;&nbsp;&nbsp;2c. Commit Changes

In [13]:
try:
    # Create Table for CSV input
    sqlTableCreate = """
    CREATE TABLE [dbo].[tblCustomersFromCSV](
        [id] [int] NOT NULL,
        [code] [varchar](50) NULL,
        [firstName] [varchar](50) NULL,
        [lastName] [varchar](50) NULL        
    )
    """
    # execute and commit cursor
    cursor.execute(sqlTableCreate) 
    connection.commit()

    # display table
    cursor.execute('SELECT * FROM tblCustomersFromCSV')
    
    print("[Before INSERT.....]")
    while 1:
        row = cursor.fetchone()
        if not row:
            break
        print(row.id, row.code, row.firstName, row.lastName)
        
    print()
    print("[Loading CSV and INSERTING Data.....]")
    # Read CSV and INSERT Rows into SQL Server Table
    with open('sample-csv-file-for-demo.csv',newline='') as csvfile:
        csvReader = csv.reader(csvfile, delimiter=';', quotechar='|')
        for row in csvReader:
            # Extract specific fields from the CSV row.
            paramID = row[0]        # Extract the ID from the first column.
            paramCode = row[1]      # Extract the code from the second column.
            paramFName = row[2]     # Extract the first name from the fourth column.
            paramLName = row[3]     # Extract the last name from the fifth column.
            
            # Print the current row's data being inserted.
            print('Inserting record: ',', '.join(row))
            
            # Execute an SQL command to insert the data into a table.
            cursor.execute("INSERT INTO [tblCustomersFromCSV] ([id], [code], [firstName], [lastName]) VALUES (?, ?, ?, ?)", 
                        paramID, paramCode, paramFName, paramLName)
            
            # Commit the transaction to save the insert operation to the database.
            connection.commit()
            
    cursor.execute('SELECT * FROM tblCustomersFromCSV')
    
    print()
    print("[After INSERT.....]")
    while 1:
        row = cursor.fetchone()
        if not row:
            break
        print(row.id, row.code, row.firstName, row.lastName)
    
except pyodbc.Error as ex: # ex is the exception object
    print()
    print('Exception: ', ex)
    print('Stopping program...')
    print()

[Before INSERT.....]

[Loading CSV and INSERTING Data.....]
Inserting record:  1, customer1, firstName1, lastName1
Inserting record:  2, customer2, firstName2, lastName2
Inserting record:  3, customer3, firstName3, lastName3
Inserting record:  4, customer4, firstName4, lastName4
Inserting record:  5, customer5, firstName5, lastName5

[After INSERT.....]
1 customer1 firstName1 lastName1
2 customer2 firstName2 lastName2
3 customer3 firstName3 lastName3
4 customer4 firstName4 lastName4
5 customer5 firstName5 lastName5
